In [ ]:

# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
from sklearn.cluster import KMeans 
from sklearn.decomposition import PCA
import collections
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# reading csv file
df_dep=pd.read_csv("../Datasets/Depression.csv")
df_anx=pd.read_csv("../Datasets/Anxiety.csv")
df_str=pd.read_csv("../Datasets/Stress.csv")

In [ ]:
# sampling dataset
df_dep_samp=df_dep.sample(n=2500)
df_anx_samp=df_anx.sample(n=2500)
df_str_samp=df_str.sample(n=2500)
X_dep=df_dep_samp.iloc[:,0:7]
X_anx=df_anx_samp.iloc[:,0:7]
X_str=df_str_samp.iloc[:,0:7]

In [ ]:
# Reduce dimensions using PCA
pca = PCA(n_components = 3, random_state=1)
X_pca_dep = pca.fit_transform(X_dep)
X_pca_anx = pca.fit_transform(X_anx)
X_pca_str = pca.fit_transform(X_str)

In [ ]:
# Kmeans

In [ ]:
### Elbow Method
wcss_dep = []
wcss_anx = []
wcss_str = []
clusters=7
for i in range(1,clusters+1):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 66)
    kmeans.fit(X_dep)
    wcss_dep.append(kmeans.inertia_)
    kmeans.fit(X_anx)
    wcss_anx.append(kmeans.inertia_)
    kmeans.fit(X_str)
    wcss_str.append(kmeans.inertia_)
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1,clusters+1)), y=wcss_dep, mode='lines+markers', name='Depression Dataset'))
fig.add_trace(go.Scatter(x=list(range(1,clusters+1)), y=wcss_anx, mode='lines+markers', name='Anxiety Dataset'))
fig.add_trace(go.Scatter(x=list(range(1,clusters+1)), y=wcss_str, mode='lines+markers', name='Stress Dataset'))
fig.show()

In [ ]:
## Kmeans visualization
kmeans = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 1)
ymeans = kmeans.fit_predict(X_dep)
print("Depression",collections.Counter(ymeans))
x, y, z = X_pca_dep.T
fig = px.scatter_3d(x = x, y = y, z = z, color = ymeans, opacity= 0.70, template='ggplot2')
fig.update_traces(marker=dict(size=4), selector=dict(mode='markers'))
fig.show()
